In [2]:
import os
import re

# === 경로 설정 (상대경로 기준) ===
base_dir = os.path.dirname("C://WORK_JANG/JSY자동화/▶ 250514_PC디렉트/GIT 업로드")
base_dir = os.getcwd()
report_dir = os.path.join(base_dir, "report_pcd")
os.chdir(base_dir)

# === report_pcd 폴더 내 리포트 파일 수집 ===
html_files = sorted(
    [f for f in os.listdir("report_pcd")
     if f.endswith(".html")],
    reverse=True
)

if not html_files:
    print("⚠️ report_pcd 폴더에 HTML 리포트가 없습니다.")
    raise SystemExit

latest_files = html_files
latest_default = latest_files[0]

# === YYYYMMDD만 버튼 라벨로 추출 ===
def yyyymmdd(name: str) -> str:
    m = re.search(r"(\d{8})", name)
    return m.group(1) if m else name

# === archive.html 생성 ===
archive_html = """<html>
<head>
<meta charset="UTF-8">
<meta name="robots" content="noindex, nofollow">
<title>Report Archive</title>
<style>
  .shell { font-family: Arial, sans-serif; margin: 20px; }
  .shell ul { line-height: 1.8; }
  .shell h1 { font-size: 20px; margin: 0 0 10px 0; }
  .shell a { color: #007bff; text-decoration: none; }
  .shell a:hover { text-decoration: underline; }
</style>
</head>
<body class="shell">
<h1>리포트 목록</h1>
<ul>
"""
for f in html_files:
    archive_html += f'  <li><a href="report_pcd/{f}">{f}</a></li>\n'
archive_html += """</ul>
</body>
</html>
"""

with open("archive.html", "w", encoding="utf-8") as f:
    f.write(archive_html)
print("✅ archive.html 생성 완료")

# === index.html 생성 (iframe 반응형) ===
buttons_html = "".join(
    [f"<button onclick=\"showReport('report_pcd/{f}')\">{yyyymmdd(f)}</button>"
     for f in latest_files]
)

index_html = f"""<html>
<head>
<meta charset="UTF-8">
<meta name="robots" content="noindex, nofollow">
<title>리포트 전체</title>
<style>
  html, body {{
    height: 100%;
    margin: 0;
  }}
  .shell {{
    font-family: Arial, sans-serif;
    background:#f9fafc;
    height: 100%;
    display: flex;
    flex-direction: column;
  }}
  .header-line {{
    display:flex; justify-content:space-between; align-items:center;
    padding: 10px 20px; background:#fff; border-bottom:1px solid #ddd;
  }}
  .header-line h1 {{ font-size:20px; margin:0; color:#222; }}
  .header-line a {{ font-size:14px; text-decoration:none; color:#007bff; }}
  .header-line a:hover {{ text-decoration:underline; }}
  .btns {{
    flex-shrink: 0;
    padding: 10px 20px;
    background:#fff;
    border-bottom:1px solid #ddd;
  }}
  .btns button {{
    margin:4px; padding:6px 10px; border-radius:5px;
    border:1px solid #bbb; background:#fff; cursor:pointer; font-size:13px;
  }}
  .btns button:hover {{ background:#f0f0f0; }}
  .btns button.active {{ background:#007bff; color:#fff; border-color:#007bff; }}
  #reportFrame {{
    flex: 1;
    width:100%;
    border: none;
  }}
</style>
<script>
function resizeFrame() {{
  const frame = document.getElementById('reportFrame');
  frame.style.height = window.innerHeight - 
    (document.querySelector('.header-line').offsetHeight +
     document.querySelector('.btns').offsetHeight + 10) + 'px';
}}
window.addEventListener('resize', resizeFrame);

function showReport(url) {{
  var btns = document.querySelectorAll('.btns button');
  for (var i=0;i<btns.length;i++) btns[i].classList.remove('active');
  for (var i=0;i<btns.length;i++) if (btns[i].getAttribute('onclick').includes(url)) btns[i].classList.add('active');
  document.getElementById('reportFrame').src = url;
  document.getElementById('fallbackLink').href = url;
  resizeFrame();
}}
window.onload = function() {{
  showReport('report_pcd/{latest_default}');
}};
</script>
</head>
<body class="shell">
  <div class="header-line">
    <h1>리포트 전체</h1>
    <a href="archive.html">리포트 목록</a>
  </div>

  <div class="btns">{buttons_html}</div>

  <iframe id="reportFrame" src="" frameborder="0"></iframe>
  <p style="margin:8px 20px;">
    열리지 않으면 <a id="fallbackLink" href="report_pcd/{latest_default}" target="_blank">새 탭으로 열기</a>
  </p>
</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(index_html)

print(f"✅ index.html 생성 완료 (전체 {len(latest_files)}개 + 창 크기 자동 맞춤)")


✅ archive.html 생성 완료
✅ index.html 생성 완료 (전체 14개 + 창 크기 자동 맞춤)


In [ ]:

https://github.com/bullazure/report_pcd.hv-stone
    


In [6]:
import subprocess
from pathlib import Path
from datetime import datetime

# ✅ 1. 여기만 본인 환경에 맞게 수정하세요
LOCAL_REPO_PATH = r"C://WORK_JANG\JSY자동화/▶ 250514_PC디렉트/GIT 업로드"  # 깃 루트 폴더 (index.html, report_pcd 폴더가 있는 위치)
REMOTE_URL      = "https://github.com/bullazure/report_pcd.hv-stone.git"            # GitHub 리포지토리 URL
BRANCH_NAME     = "main"                                                            # 현재 GitHub 기본 브랜치
COMMIT_MESSAGE  = None                                                              # None이면 자동 생성


def run_git(cmd, cwd):
    """git 명령어 실행 헬퍼 함수"""
    print("$", " ".join(cmd))
    result = subprocess.run(
        cmd,
        cwd=cwd,
        text=True,
        capture_output=True,
        shell=False
    )
    if result.stdout:
        print(result.stdout)
    if result.returncode != 0:
        print("[ERROR]", result.stderr)
    return result


repo_path = Path(LOCAL_REPO_PATH)

if not repo_path.exists():
    raise FileNotFoundError(f"로컬 경로가 존재하지 않습니다: {repo_path}")

# 0) .git 유무 확인
git_dir = repo_path / ".git"

if not git_dir.exists():
    # Git 초기화 + 원격 연결 (처음 세팅할 때만 실행됨)
    print("🔧 .git 폴더가 없어 Git 저장소를 초기화합니다.")
    run_git(["git", "init"], cwd=repo_path)

    # origin 재설정
    run_git(["git", "remote", "remove", "origin"], cwd=repo_path)
    run_git(["git", "remote", "add", "origin", REMOTE_URL], cwd=repo_path)

    # 브랜치 이름 강제(main)
    run_git(["git", "branch", "-M", BRANCH_NAME], cwd=repo_path)
else:
    print("✅ 이미 Git 저장소입니다.")

    # origin이 설정되어 있는지 확인하고, 없으면 추가
    remotes = run_git(["git", "remote"], cwd=repo_path)
    if "origin" not in remotes.stdout.split():
        print("🔗 origin 원격이 없어 새로 추가합니다.")
        run_git(["git", "remote", "add", "origin", REMOTE_URL], cwd=repo_path)

# 1) 원격 변경 사항 먼저 가져오기(충돌 방지)
print("📥 원격 변경 사항을 가져옵니다 (git pull)...")
run_git(["git", "pull", "origin", BRANCH_NAME, "--rebase"], cwd=repo_path)

# 2) 변경 사항 전체 스테이징
print("📂 변경된 파일을 모두 git add 합니다...")
run_git(["git", "add", "-A"], cwd=repo_path)

# 3) 커밋 메시지 생성
if COMMIT_MESSAGE is None:
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    commit_msg = f"auto update: {now}"
else:
    commit_msg = COMMIT_MESSAGE

print(f"📝 커밋 메시지: {commit_msg}")

commit_result = run_git(["git", "commit", "-m", commit_msg], cwd=repo_path)

# 4) 커밋이 있을 때만 push
if commit_result.returncode != 0:
    msg = (commit_result.stdout + commit_result.stderr).lower()
    if "nothing to commit" in msg:
        print("ℹ️ 커밋할 변경 사항이 없습니다. push를 생략합니다.")
    else:
        print("❌ commit 중 문제가 발생했습니다. 위 에러 메시지를 확인하세요.")
else:
    print(f"🚀 origin/{BRANCH_NAME} 으로 push 합니다...")
    push_result = run_git(["git", "push", "-u", "origin", BRANCH_NAME], cwd=repo_path)

    if push_result.returncode == 0:
        print("🎉 GitHub로 정상 push 완료!")
    else:
        print("❌ push 중 문제가 발생했습니다. 위 에러 메시지를 확인하세요.")


🔧 .git 폴더가 없어 Git 저장소를 초기화합니다.
$ git init


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [ ]:
current_directory = os.getcwd()
print(current_directory)

In [ ]:
base_dir = os.path.dirname("C://WORK_JANG/JSY자동화/▶ 250514_PC디렉트/GIT 업로드")
# base_dir = os.getcwd()
report_dir = os.path.join(base_dir, "report_pcd")

In [ ]:
report_dir

In [4]:
os.listdir('C://WORK_JANG/JSY자동화/▶ 250514_PC디렉트/report_pcd')

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C://WORK_JANG/JSY자동화/▶ 250514_PC디렉트/report_pcd'

In [6]:
os.listdir("report_pcd")

['2025_0825.html',
 '2025_0908.html',
 '2025_0915.html',
 '2025_0922.html',
 '2025_0930.html',
 '2025_1014.html',
 '2025_1021.html',
 '2025_1027.html',
 '[헤비스톤] 브랜드별 주간 평판 모니터링_20251104.html']